# Task A 

In [ ]:

# Approach: Floyd's Tortoise and Hare algorithm

class ListNode:
    def __init__(self, x):
        self.val = x
        self.next = None

class Solution:
    def detectCycle(self, head):
        slow = head
        fast = head

        # Step 1: Detect cycle
        while fast and fast.next:
            slow = slow.next
            fast = fast.next.next

            if slow == fast:
                break
        else:
            return None   # no cycle

        # Step 2: Find cycle start
        slow = head
        while slow != fast:
            slow = slow.next
            fast = fast.next

        return slow


# TASK-B

In [ ]:
import sys
import requests
import json
import os
API_KEY=os.getenv("GEMINI")

URL = "https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash:generateContent"
    
prompt = "what is AI"

payload = {
    "contents": [
        {"parts": [{"text": prompt}]}
    ]
}

res = requests.post(f"{URL}?key={API_KEY}", json=payload)
data = res.json()

if "candidates" in data:
    print("---- RESPONSE ----")
    print(data["candidates"][0]["content"]["parts"][0]["text"])
else:
    print("---- API ERROR ----")
    print(json.dumps(data, indent=2))



---- RESPONSE ----
**Artificial Intelligence (AI)** is a broad field of computer science dedicated to creating machines that can perform tasks that typically require human intelligence.

In simpler terms, **AI is about making computers "smart" like humans, enabling them to learn, reason, solve problems, perceive, and understand language.**

Here's a breakdown of what that means:

1.  **Learning:** AI systems can learn from data, identifying patterns and making decisions or predictions based on those patterns. This is often achieved through **Machine Learning (ML)**, where algorithms are "trained" on vast amounts of data.
    *   *Example:* An AI system trained on millions of images of cats and dogs can learn to identify new images as either a cat or a dog.

2.  **Reasoning:** AI can process information and apply logic to solve problems or make inferences, much like humans do.
    *   *Example:* A chess AI can analyze the board state and determine the best move based on its learned stra

# TASK-C

In [71]:
# optional 


from glob import glob
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from dotenv import load_dotenv
load_dotenv()
import os
os.environ["GOOGLE_API_KEY"]=os.getenv("GEMINI")


docs = []
for path in glob(r"C:\Users\Windows\Downloads\GEN_AI Interview questions.pdf", recursive=True):
    docs.extend(PyPDFLoader(path).load())

splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = splitter.split_documents(docs)

print(f"Loaded {len(chunks)} chunks")

from langchain_community.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="intfloat/e5-small-v2")


from langchain_community.vectorstores import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings
emb = GoogleGenerativeAIEmbeddings(model="intfloat/e5-small-v2",task_type="retrieval_document")
vectordb = Chroma.from_documents(chunks, 
                                 embedding=embeddings,
                                 )

from langchain_google_genai import ChatGoogleGenerativeAI
llm= ChatGoogleGenerativeAI(model="gemini-2.5-flash")


def simple_rag(question):
    docs = vectordb.similarity_search_with_score(question, k=3)

    if not docs:
        return "I don't have that information in the document."

    # docs is now: [(Document, score), (Document, score)…]
    top_doc, score = docs[0]

    if score > 0.7:   # Chroma uses LOWER score = better
        return "I don't have that information in the document."

    context = "\n\n".join([doc.page_content for doc, _ in docs])

    prompt = f"""You are a QA bot.

Answer ONLY using the context below.
If the answer is not found, say:
"I don't have that information in the document."

Context:
{context}

Question: {question}
"""

    response = llm.invoke(prompt)
    return response.content.strip()


# Usage
question = "what is Generative AI?"
answer = simple_rag(question)
print("Q:", question)
print("A:", answer)


Loaded 132 chunks
Q: what is Generative AI?
A: Generative AI is a subset of artificial intelligence that creates new content from learned patterns in training data. Unlike traditional AI that analyzes or classifies data, generative AI produces original text, images, code, music, or other content. It works by learning statistical patterns from vast datasets and using these patterns to generate contextually relevant outputs.


In [68]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer("intfloat/e5-small-v2")

sentences = [
    "Machine learning models learn patterns from data.",
    "The capital of India is New Delhi.",
    "GenAI models like Gemini and ChatGPT generate text.",
    "RAG improves answers using external documents."
]

embeddings = model.encode(sentences, convert_to_tensor=True)

query = "How does RAG work?"
query_emb = model.encode(query, convert_to_tensor=True)

scores = util.cos_sim(query_emb, embeddings)[0]

best_index = scores.argmax().item()

print("Query:", query)
print("Most relevant sentence:", sentences[best_index])


Query: How does RAG work?
Most relevant sentence: RAG improves answers using external documents.
